# Embeddings

Решим задачу семантической классификации твитов.  
Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
import gdown

In [2]:
gdown.download('https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download')

In [3]:
!unzip archive.zip

Импортируем нужные библиотеки:

In [4]:
import re
import math
import torch
import nltk
import gensim
import random
import string
import numpy as np
import pandas as pd
import gensim.downloader as api
from torch.utils.data import Dataset, random_split

%matplotlib inline
from tqdm import tqdm

Зафиксируем `random`

In [5]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

Определим `device`

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", 
                   encoding="latin", 
                   header=None, 
                   names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [8]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем некоторые статистики:

In [9]:
data.describe(include="all")

,emotion,id,date,flag,user,text
count,1.600000e+06,1.600000e+06,1600000,1600000,1600000,1600000
unique,NaN,NaN,774363,1,659775,1581466
top,NaN,NaN,Mon Jun 15 12:53:14 PDT 2009,NO_QUERY,lost_dog,isPlayer Has Died! Sorry
freq,NaN,NaN,20,1600000,549,210
mean,2.000000e+00,1.998818e+09,NaN,NaN,NaN,NaN
std,2.000001e+00,1.935761e+08,NaN,NaN,NaN,NaN
min,0.000000e+00,1.467810e+09,NaN,NaN,NaN,NaN
25%,0.000000e+00,1.956916e+09,NaN,NaN,NaN,NaN
50%,2.000000e+00,2.002102e+09,NaN,NaN,NaN,NaN
75%,4.000000e+00,2.177059e+09,NaN,NaN,NaN,NaN


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [10]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [11]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [12]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [13]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 
Данная модель выдает эмбеддинги для **слов**.

In [14]:
word2vec = api.load("word2vec-google-news-300")

In [15]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги:

In [16]:
mean = np.mean(word2vec.vectors, axis=0)
std = np.std(word2vec.vectors, axis=0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [17]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получим все токены из текста и профильтруем их
        text = re.sub(r'@\S+', '', text)           # уберём ники вида @chrishasboobs
        text = re.sub(r'http\S+', '', text)        # уберём http ссылки
        text = re.sub('[^A-Za-z0-9]+', ' ', text)  # уберём слова в неверной кодировке
        # заменим все символы, которые повторяются >3 раз подряд на единичные (stoppp -> stop)
        text = re.sub(r'([A-Za-z0-9])\1(?=\1)', '', text)  
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line

    def get_embeddings_(self, tokens):
        # Получим эмбеддинги слов и нормализуем их
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [18]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

## Average embedding (2 балла)
---
Вектор предложения есть средний вектор всех слов в предложeнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [19]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета:

In [20]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_jobs=-1)
examples["transformed_features"] = tsne.fit_transform(examples["features"])  # Обучим TSNE на эмбеддингах слов

In [21]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=2)
# examples["transformed_features"] = pca.fit_transform(examples["features"])  # Обучим PCA на эмбеддингах слов

In [22]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [23]:
draw_vectors(examples["transformed_features"][:, 0],
             examples["transformed_features"][:, 1],
             color=[["red", "blue"][t] for t in examples["targets"]])

Figure(id='1004', ...)

На визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднение векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [24]:
from torch.utils.data import DataLoader


batch_size = 1024
# num_workers = 4

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch])

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [25]:
from sklearn.metrics import accuracy_score


def training(model, optimizer, criterion, train_loader, e, device=device):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}, Train Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        
        optimizer.zero_grad()

        preds = model(features)                                 # Получим предсказания модели
        loss = criterion(preds, targets)                        # Посчитаем лосс
        acc = accuracy_score(targets.cpu(), preds.cpu() > 0.5)  # Посчитаем точность модели
        
        mean_loss += loss.item()
        mean_acc += acc.item()
    
        # Обновим параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}, Train Acc: {acc:.4}")
    
    return {"Train Loss": mean_loss / len(train_loader), "Train Acc": mean_acc / len(train_loader)}
    

def testing(model, criterion, test_loader, device=device):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            preds = model(features)                                 # Получим предсказания модели
            loss = criterion(preds, targets)                        # Посчитаем лосс
            acc = accuracy_score(targets.cpu(), preds.cpu() > 0.5)  # Посчитаем точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию.

In [26]:
import torch.nn as nn

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight)

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 10

model = nn.Sequential(nn.Linear(in_features=vector_size, out_features=vector_size),
                      nn.ReLU(),
                      nn.Linear(in_features=vector_size, out_features=1),
                      nn.Sigmoid(),
                      nn.Flatten(start_dim=0))

model.apply(init_normal)
model = model.to(device)
criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета.

In [27]:
best_metric = np.inf
for e in range(num_epochs):
    train_log = training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(train_log)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Test Loss: 0.4802, Test Acc: 0.7607: 100%|██████████| 250/250 [00:34<00:00,  7.22it/s]


{'Train Loss': 1.1274324446618558, 'Train Acc': 0.7320029296875}
{'Test Loss': 0.5172797170877457, 'Test Acc': 0.75400390625}


Test Loss: 0.4736, Test Acc: 0.7578: 100%|██████████| 250/250 [00:35<00:00,  7.10it/s]


{'Train Loss': 0.5088474698662758, 'Train Acc': 0.7552099609375}
{'Test Loss': 0.5000405178070069, 'Test Acc': 0.75967578125}


Test Loss: 0.4752, Test Acc: 0.7539: 100%|██████████| 250/250 [00:35<00:00,  7.09it/s]


...

{'Train Loss': 0.4851794149577618, 'Train Acc': 0.7671484375}
{'Test Loss': 0.49004113841056823, 'Test Acc': 0.76766015625}


Test Loss: 0.4652, Test Acc: 0.7607: 100%|██████████| 250/250 [00:33<00:00,  7.36it/s]


{'Train Loss': 0.48337178406119347, 'Train Acc': 0.767890625}
{'Test Loss': 0.4853319125175476, 'Test Acc': 0.7686953125}


Test Loss: 0.4679, Test Acc: 0.7568: 100%|██████████| 250/250 [00:34<00:00,  7.34it/s]

{'Train Loss': 0.4823636828958988, 'Train Acc': 0.7680234375}
{'Test Loss': 0.4899590277671814, 'Test Acc': 0.76875}


In [28]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0.5879, Test Acc: 0.7324: 100%|██████████| 313/313 [00:42<00:00,  7.40it/s]

{'Test Loss': 0.490078897997975, 'Test Acc': 0.7657966004392971}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста.
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf.

**Вариант 1**

Сформируем класс датасета с учётом контекста. В качестве контекста возмём `3` слова с каждой стороны от незнакомого слова.

In [27]:
class TwitterDataset_context(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text = re.sub(r'@\S+', '', text)           # уберём ники вида @chrishasboobs
        text = re.sub(r'http\S+', '', text)        # уберём http ссылки
        text = re.sub('[^A-Za-z0-9]+', ' ', text)  # уберём слова в неверной кодировке
        # заменим все символы, которые повторяются >3 раз подряд на единичные (stoppp -> stop) 
        text = re.sub(r'([A-Za-z0-9])\1(?=\1)', '', text)  
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line
    
    def get_vector_(self, token):
        return (self.word2vec.get_vector(token) - self.mean) / self.std

    def get_embeddings_(self, tokens):
        window_size = 3
        embeddings = []                                          
        
        for i, token in enumerate(tokens):
            if token in self.word2vec:
                embeddings.append(self.get_vector_(token))
            else:
                context = tokens[max(i-window_size, 0):min(i+window_size, len(tokens))]
                context.remove(token)
                context_embedding = np.sum(np.array([self.get_vector_(w) for w in context if w in self.word2vec]), axis=0)
                if context_embedding.all() == 0:
                    continue
                embeddings.append(context_embedding)
                
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

Сформируем датасет:

In [28]:
dev_context = TwitterDataset_context(dev_data, "text", "emotion", word2vec)

Разобъём датасет на тренировочную и валидационную части и создадим даталоадеры:

In [29]:
train_size = math.ceil(len(dev_context) * 0.8)

train, valid = random_split(dev_context, [train_size, len(dev_context) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Создадим модель:

In [30]:
model = nn.Sequential(nn.Linear(in_features=vector_size, out_features=vector_size),
                      nn.ReLU(),
                      nn.Linear(in_features=vector_size, out_features=1),
                      nn.Sigmoid(),
                      nn.Flatten(start_dim=0))

model.apply(init_normal)
model = model.to(device)
criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

Проведём обучение модели:

In [33]:
best_metric = np.inf
for e in range(num_epochs):
    train_log = training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(train_log)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_context.pt")
        best_metric = log["Test Loss"]

Test Loss: 0.5056, Test Acc: 0.7363: 100%|██████████| 250/250 [00:41<00:00,  6.07it/s]


{'Train Loss': 1.1624080924093723, 'Train Acc': 0.730255859375}
{'Test Loss': 0.5437137211561203, 'Test Acc': 0.74480859375}


Test Loss: 0.5061, Test Acc: 0.752: 100%|██████████| 250/250 [00:41<00:00,  5.95it/s] 


{'Train Loss': 0.5221134818196297, 'Train Acc': 0.7504248046875}
{'Test Loss': 0.5169343272447586, 'Test Acc': 0.7504375}


Test Loss: 0.5092, Test Acc: 0.7529: 100%|██████████| 250/250 [00:41<00:00,  6.06it/s]


...

{'Train Loss': 0.4954324324131012, 'Train Acc': 0.7619189453125}
{'Test Loss': 0.5138514807224274, 'Test Acc': 0.75469140625}


Test Loss: 0.492, Test Acc: 0.7637: 100%|██████████| 250/250 [00:41<00:00,  6.08it/s] 


{'Train Loss': 0.49633976665139196, 'Train Acc': 0.7625283203125}
{'Test Loss': 0.5029434760808945, 'Test Acc': 0.7585625}


Test Loss: 0.4968, Test Acc: 0.7656: 100%|██████████| 250/250 [00:41<00:00,  6.06it/s]

{'Train Loss': 0.4939254277944565, 'Train Acc': 0.7628408203125}
{'Test Loss': 0.5072914344072342, 'Test Acc': 0.75830859375}


Проверим качество модели на тестовых данных:

In [34]:
test_loader = DataLoader(
    TwitterDataset_context(test_data, "text", "emotion", word2vec),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_context.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0.6046, Test Acc: 0.7188: 100%|██████████| 313/313 [00:51<00:00,  6.05it/s]

{'Test Loss': 0.5028491101135462, 'Test Acc': 0.7595129043530351}


**Вариант 2**

Токенизируем полный датасет в целях подготовки данных для `TfidfVectorizer`, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [35]:
tokenized_data = []
tokenizer = nltk.WordPunctTokenizer()

for i in tqdm(range(len(data))):
    text = data["text"][i]
    text = re.sub(r'@\S+', '', text)           # уберём ники вида @chrishasboobs
    text = re.sub(r'http\S+', '', text)        # уберём http ссылки
    text = re.sub('[^A-Za-z0-9]+', ' ', text)  # уберём слова в неверной кодировке
    # заменим все символы, которые повторяются >3 раз подряд на единичные (stoppp -> stop)
    text = re.sub(r'([A-Za-z0-9])\1(?=\1)', '',text)
    line = tokenizer.tokenize(text.lower())    # приведём к нижнему регистру
    filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
    filtered_string = " ".join(filtered_line)
    tokenized_data.append(filtered_string)  

100%|██████████| 1600000/1600000 [00:59<00:00, 26816.64it/s]


In [36]:
len(tokenized_data)

1600000

Выведем несколько строк подготовленных данных:

In [37]:
for i in range(10):
    print(tokenized_data[i])

that bummer you shoulda got david carr third day
upset that can update his facebook texting and might cry result school today also blah
dived many times for the ball managed save the rest out bounds
whole body feels itchy and like its fire
not behaving all mad why here because can see you all over there
not the whole crew
need hug
hey long time see yes rains bit only bit lol fine thanks how you
nope they didn have
que muera


Обучим `TfidfVectorizer` на полном датасете:

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(lowercase=False)
vectors = vectorizer.fit_transform(tokenized_data)
vectors.shape

(1600000, 257832)

Так как оперируем векторами размера `300`, выполним `SVD` преобразование для уменьшения размерности:

In [39]:
from sklearn.decomposition import TruncatedSVD
 
trun_svd = TruncatedSVD(n_components=300)
vectors_transformed = trun_svd.fit_transform(vectors)
vectors_transformed.shape

(1600000, 300)

Подготовим класс датасета:

In [40]:
class TwitterDataset_tfidf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, 
                 word2vec: gensim.models.Word2Vec, tfidf, t_vectors: np.array):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf = tfidf
        self.t_vectors = t_vectors

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        self.mean_tfidf = np.mean(t_vectors, axis=0)
        self.std_tfidf = np.std(t_vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text = re.sub(r'@\S+', '', text)           # уберём ники вида @chrishasboobs
        text = re.sub(r'http\S+', '', text)        # уберём http ссылки
        text = re.sub('[^A-Za-z0-9]+', ' ', text)  # уберём слова в неверной кодировке
        # заменим все символы, которые повторяются >3 раз подряд на единичные (stoppp -> stop)
        text = re.sub(r'([A-Za-z0-9])\1(?=\1)', '', text)
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line
    
    def get_w2v_vector_(self, token):
        return (self.word2vec.get_vector(token) - self.mean) / self.std
    
    def get_tfidf_vector_(self, token):
        tfidf_idx = self.tfidf.vocabulary_[token]
        tfidf_vector = self.t_vectors[tfidf_idx]
        return (tfidf_vector - self.mean_tfidf) / self.std_tfidf

    def get_embeddings_(self, tokens):
        embeddings = []
        
        for token in tokens:
            if token in self.word2vec:
                w2v_emb = self.get_w2v_vector_(token)
                tfidf_emb = self.get_tfidf_vector_(token)
                embeddings.append(w2v_emb + tfidf_emb)
            else:
                tfidf_emb = self.get_tfidf_vector_(token)
                embeddings.append(tfidf_emb)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

Сформируем датасет:

In [41]:
dev_tfidf = TwitterDataset_tfidf(dev_data, "text", "emotion", word2vec, vectorizer, vectors_transformed)

Разобъём датасет на тренировочную и валидационную части и создадим даталоадеры:

In [42]:
train_size = math.ceil(len(dev_tfidf) * 0.8)

train, valid = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Создадим модель:

In [43]:
model = nn.Sequential(nn.Linear(in_features=vector_size, out_features=vector_size),
                      nn.ReLU(),
                      nn.Linear(in_features=vector_size, out_features=1),
                      nn.Sigmoid(),
                      nn.Flatten(start_dim=0))

model.apply(init_normal)
model = model.to(device)
criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

Проведём обучение модели:

In [44]:
best_metric = np.inf
for e in range(num_epochs):
    train_log = training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(train_log)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Test Loss: 0.5387, Test Acc: 0.7354: 100%|██████████| 250/250 [00:49<00:00,  5.01it/s]


{'Train Loss': 1.2678647902607918, 'Train Acc': 0.718419921875}
{'Test Loss': 0.6072666885852813, 'Test Acc': 0.73495703125}


Test Loss: 0.5129, Test Acc: 0.7529: 100%|██████████| 250/250 [00:49<00:00,  5.07it/s]


{'Train Loss': 0.5442264665961265, 'Train Acc': 0.7409833984375}
{'Test Loss': 0.5295983834266662, 'Test Acc': 0.74490625}


Test Loss: 0.5097, Test Acc: 0.7578: 100%|██████████| 250/250 [00:49<00:00,  5.06it/s]


...

{'Train Loss': 0.5140500944554806, 'Train Acc': 0.7573525390625}
{'Test Loss': 0.54930238199234, 'Test Acc': 0.7553125}


Test Loss: 0.4943, Test Acc: 0.7646: 100%|██████████| 250/250 [00:49<00:00,  5.01it/s]


{'Train Loss': 0.5157421705722809, 'Train Acc': 0.7583408203125}
{'Test Loss': 0.5327710771560669, 'Test Acc': 0.755296875}


Test Loss: 0.5034, Test Acc: 0.7627: 100%|██████████| 250/250 [00:49<00:00,  5.00it/s]

{'Train Loss': 0.5126367146968842, 'Train Acc': 0.7591826171875}
{'Test Loss': 0.5357319663763046, 'Test Acc': 0.756015625}


Проверим качество модели на тестовых данных:

In [45]:
test_loader = DataLoader(
    TwitterDataset_tfidf(test_data, "text", "emotion", word2vec, vectorizer, vectors_transformed),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0.5745, Test Acc: 0.7363: 100%|██████████| 313/313 [01:02<00:00,  5.02it/s]

{'Test Loss': 0.5289415219150031, 'Test Acc': 0.7536722494009584}


## Выводы:

Для адекватного сравнения была выполнена идентичная токенизация для всех трёх моделей. А именно, выполнена следующая фильтрация:
- удалены ники пользователей
- удаленны http ссылки на web ресурсы
- удалены все слова и символы в кодировках отличных от исходной
- символы, которые повторяются более 3 раз заменены на единичные (stoppp -> stop)
- текст приведён к нижнему регистру
- удалены знаки пунктуации
- удалены все слова состоящие менее из 3-х символов  

В эти условиях наилучшее качество показала базовая модель ('Test Acc': 0.765). Следом за ней по точности идёт модель, в которой эмбеддингом незнакомого слова будет сумма эмбеддингов всех слов из его контекста ('Test Acc': 0.758). Хуже всех показала себя модель с применением эмбеддингов из Tfidf ('Test Acc': 0.753). В данном случае дополнительные техники обработки слов, которых нет в словаре, не принесли пользы. Возможно, суммирование контекста и добавление эмбеддингов Tfidf только вносят искажения, тем самым негативно влияя на результат. Потому что норма суммы векторов контекста для незнакомого слова будет значительно выше нормы вектора известного слова (вариант 1), а норма известных слов (вариант 2) полученной в виде суммы обычного эмбеддинга и tfidf будет больше нормы tfidf вектора неизвестных слова. Не говоря уже о том, что tfidf векторизованы по другому принципу.
Тем не менее, значения точностей всех трёх моделей близки. 


Стоит отметить, что при изменении принципов токенизации и фильтрации токенов, можно добиться иного распределения результатов между моделями, особенно если настраивать эти параметры для каждой модели в отдельности. Но я ставил цель сравнения моделей в одинаковых условиях.

Исправим недостатки предыдущих подходов. Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет СРЕДНЕЕ эмбеддингов всех слов из его контекста.

In [52]:
class TwitterDataset_context_mean(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text = re.sub(r'@\S+', '', text)           # уберём ники вида @chrishasboobs
        text = re.sub(r'http\S+', '', text)        # уберём http ссылки
        text = re.sub('[^A-Za-z0-9]+', ' ', text)  # уберём слова в неверной кодировке
        # заменим все символы, которые повторяются >3 раз подряд на единичные (stoppp -> stop) 
        text = re.sub(r'([A-Za-z0-9])\1(?=\1)', '', text)  
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 2]
        
        return filtered_line
    
    def get_vector_(self, token):
        return (self.word2vec.get_vector(token) - self.mean) / self.std

    def get_embeddings_(self, tokens):
        window_size = 3
        embeddings = []                                          
        
        for i, token in enumerate(tokens):
            if token in self.word2vec:
                embeddings.append(self.get_vector_(token))
            else:
                context = tokens[max(i-window_size, 0):min(i+window_size, len(tokens))]
                context.remove(token)
                context_embeddings = np.array([self.get_vector_(w) for w in context if w in self.word2vec])
                context_embedding = np.sum(context_embeddings, axis=0)
                if context_embedding.all() == 0:
                    continue
                embeddings.append(context_embedding / len(context_embeddings))
                
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

Сформируем датасет:

In [53]:
dev_context_mean = TwitterDataset_context_mean(dev_data, "text", "emotion", word2vec)

Разобъём датасет на тренировочную и валидационную части и создадим даталоадеры:

In [54]:
train_size = math.ceil(len(dev_context_mean) * 0.8)

train, valid = random_split(dev_context_mean, [train_size, len(dev_context_mean) - train_size])

train_loader = DataLoader(train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True, 
                          collate_fn=average_emb)

valid_loader = DataLoader(valid, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          drop_last=False, 
                          collate_fn=average_emb)

Создадим модель:

In [55]:
model = nn.Sequential(nn.Linear(in_features=vector_size, out_features=vector_size),
                      nn.ReLU(),
                      nn.Linear(in_features=vector_size, out_features=1),
                      nn.Sigmoid(),
                      nn.Flatten(start_dim=0))

model.apply(init_normal)
model = model.to(device)
criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

Проведём обучение модели:

In [56]:
best_metric = np.inf
for e in range(num_epochs):
    train_log = training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(train_log)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_context_mean.pt")
        best_metric = log["Test Loss"]

Test Loss: 0.5943, Test Acc: 0.7627: 100%|██████████| 250/250 [00:40<00:00,  6.15it/s]


{'Train Loss': 1.0940055283606052, 'Train Acc': 0.730822265625}
{'Test Loss': 0.5169219068288803, 'Test Acc': 0.7517578125}


Test Loss: 0.5862, Test Acc: 0.7598: 100%|██████████| 250/250 [00:40<00:00,  6.18it/s]


{'Train Loss': 0.5090792016983032, 'Train Acc': 0.7549365234375}
{'Test Loss': 0.5094855724573135, 'Test Acc': 0.75663671875}


Test Loss: 0.4861, Test Acc: 0.7676: 100%|██████████| 250/250 [00:41<00:00,  6.09it/s]


...

{'Train Loss': 0.48751047483086585, 'Train Acc': 0.7658994140625}
{'Test Loss': 0.5020015186071396, 'Test Acc': 0.7627421875}


Test Loss: 0.4739, Test Acc: 0.7715: 100%|██████████| 250/250 [00:40<00:00,  6.10it/s]


{'Train Loss': 0.4867734566926956, 'Train Acc': 0.7664931640625}
{'Test Loss': 0.4941212521791458, 'Test Acc': 0.76447265625}


Test Loss: 0.4767, Test Acc: 0.7666: 100%|██████████| 250/250 [00:42<00:00,  5.92it/s]

{'Train Loss': 0.48500264251232145, 'Train Acc': 0.7672919921875}
{'Test Loss': 0.49438302135467527, 'Test Acc': 0.76441796875}


Проверим качество модели на тестовых данных:

In [58]:
test_loader = DataLoader(
    TwitterDataset_context(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size,  
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_context_mean.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0.5505, Test Acc: 0.7285: 100%|██████████| 313/313 [00:50<00:00,  6.26it/s]

{'Test Loss': 0.5032407702348484, 'Test Acc': 0.7608888278753994}


Полученное качество модели лучше чем подходы из 1 и 2 вариантов, но всё равно не дотягивает по качеству до исходного варинта без дополнительных техник обработки слов, которых нет в словаре.